## Ideas
- Create user scenarios (e.g. "want to build a family") and pre-select factors
- Recommend similar cities

# Statistics per location
1. COVID-19 Data
2. Population Data
3. Income Data
4. Occupation Data
5. Diversity Data
6. Crime
7. Weather preferences
## MISSING FINANCIAL STUFF
8. GDP per County across industries

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np

import wikipedia
from fuzzywuzzy import process, fuzz

Below is a list of incorporated U.S cities of population greater than 50,000. As well as their 2010 Census numbers and accompanying population estimates from 2010-2019.
link: https://www.census.gov/data/datasets/time-series/demo/popest/2010s-total-cities-and-towns.html

Below is a list of per capita spending by various counties across a range of industries from 1997-2019. Link: https://apps.bea.gov/regional/downloadzip.cfm

In [ ]:
per_capita_spending = pd.read_csv('data/per_capita_spending.csv')
per_capita_spending

,GeoFIPS,GeoName,Description,1997,1998,1999,2000,2001,2002,2003,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,0,United States,Per capita personal consumption expenditures,20298,21295,22490,23953,24781,25514,26603,...,32903,34124,35044,35790,37118,38331,39521,41028,42802,44276
1,0,United States,Goods,7349,7633,8184,8682,8850,9021,9366,...,10699,11261,11563,11779,12111,12212,12359,12817,13351,13712
2,0,United States,Durable goods,2624,2825,3066,3234,3304,3426,3508,...,3391,3510,3646,3764,3902,4078,4181,4341,4535,4674
3,0,United States,Motor vehicles and parts,1075,1161,1257,1287,1345,1395,1384,...,1114,1172,1264,1321,1389,1483,1504,1550,1602,1590
4,0,United States,Furnishings and durable household equipment,589,629,685,738,754,785,811,...,779,792,809,834,868,918,958,999,1051,1089
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1435,98000,Far West,Financial services and insurance,1853,1993,2085,2252,2145,2142,2209,...,2520,2625,2673,2788,2933,3083,3168,3355,3564,3725
1436,98000,Far West,Other services,1622,1730,1864,2029,2151,2202,2315,...,2739,2815,2906,2895,3029,3136,3278,3447,3650,3902
1437,98000,Far West,Final consumption expenditures of nonprofit ...,346,372,424,498,576,670,660,...,958,1020,1137,1188,1224,1299,1413,1483,1544,1560
1438,98000,Far West,Gross output of nonprofit institutions,1758,1824,1935,2095,2282,2511,2582,...,3754,3951,4249,4359,4492,4759,5035,5244,5499,5748


In [ ]:
header = cities.head()
header
#sns.scatterplot(data=header)

,city_population,city_name,state_name,matched_city_name
0,"8,336,817",New York,New York,New York
1,"3,979,576",Los Angeles,California,Los Angeles
2,"2,693,976",Chicago,Illinois,Chicago
3,"2,320,268",Houston,Texas,Houston
4,"1,680,992",Phoenix,Arizona,Phoenix


Data sets: 
- [MoveHub](https://drive.google.com/uc?id=1hSMhl-JeTCX-t72KjhasTQoL1LdWSRhw)
- [Cost of living](https://www.kaggle.com/debdutta/cost-of-living-index-by-country)
- [Zillow Rent Index](https://www.kaggle.com/zillow/rent-index)

In [3]:
covid_by_county = pd.read_csv('data/covid_data.csv')
covid_by_county = covid_by_county.rename(columns={'fips': 'county_fips'})

## First, load in data

### Population

In [ ]:
city_pops = pd.read_csv("data/city_populations.csv")
city_pops

### City-County mapping

In [ ]:
city_county_mapping = pd.read_csv("data/city_county_fips.csv")

first = lambda x: x.iloc[0]

county_info = city_county_mapping.groupby("county_fips").aggregate({
    "county_name": first,
    "state_id": first,
    "state_name": first,
    "lat": np.mean,
    "lng": np.mean,
    "population": np.sum,
    "timezone": first,
}).reset_index()
county_info

In [ ]:
census_info = pd.read_csv('data/2017_census_county.csv')
census_info

# Describing each of the weird columns above:
1. Error Columns
    - For all error columns, the error is the +- of each estimate. 
    - Example: income 55,317 and income_error 2838 means that median income is within 2838 in each direction of listed amount.
2. professional,service, office,construction, production
   - outlines the proportion of population in county that works in each of the catagories. Not exactly sure what professional is (probably similar to private practice) but service, office, construction are pretty self explanatory and production is likely ag. Will test
3. drive, carpool, transit, walk, other_t...
   - for those who commute to work, the proportion of method. Seems like one county gets 61.8% transit... Probably NYC

In [ ]:
crime_cities = pd.read_csv('data/covid_data.csv')
crime_cities.head()

## Put together a combined dataframe

### County-level information

In [ ]:
counties = pd.merge(county_info, census_info, left_on = "county_fips", right_on = "county_id")
counties.to_csv('County_Info.csv')
counties

### City-level information

In [ ]:
cities = city_pops[["city_name", "2019"]].rename(columns = {"city_name": "city_state_name", "2019": "city_population"})

cities["city_name"] = cities["city_state_name"].apply(lambda x: x.split(",")[0][:-5].strip())
cities["state_name"] = cities["city_state_name"].apply(lambda x: x.split(",")[1].strip())

cities.drop(columns = ["city_state_name"], inplace = True)

cities

## Merging city & county level info

In [ ]:
city_name = cities["city_name"].iloc[1]
city_name

cities_length = len(cities)
min_score = 75

def fuzzy_match(row):
    city_name = row["city_name"]

    print("Matching {}/{}".format(row.name, cities_length), end = "\r")

    mapping_filtered = city_county_mapping[city_county_mapping["state_name"] == row["state_name"]]

    if city_name in mapping_filtered["city_ascii"]:
        return city_name

    choice, score, _ = process.extractOne(city_name, mapping_filtered["city_ascii"])
    return choice if score > min_score else None

cities_matched = cities
cities_matched["matched_city_name"] = cities_matched.apply(fuzzy_match, axis = "columns")

cities_matched

,city_population,city_name,state_name,matched_city_name
0,"8,336,817",New York,New York,New York
1,"3,979,576",Los Angeles,California,Los Angeles
2,"2,693,976",Chicago,Illinois,Chicago
3,"2,320,268",Houston,Texas,Houston
4,"1,680,992",Phoenix,Arizona,Phoenix
...,...,...,...,...
783,"49,678",Lakewood,Ohio,Lakewood
784,"49,154",Troy,New York,Troy
785,"48,115",Saginaw,Michigan,Saginaw
786,"47,720",Niagara Falls,New York,Niagara Falls


In [ ]:
df = pd.merge(
    cities_matched,
    city_county_mapping,
    left_on = ["matched_city_name", "state_name"],
    right_on = ["city_ascii", "state_name"]
)

df = pd.merge(
    df,
    census_info,
    left_on = "county_fips",
    right_on = "county_id"
)

df["pct_men"] = df["num_men"] / df["total_pop"]
df["pct_women"] = df["num_women"] / df["total_pop"]
df["pct_voters"] = df["num_voting_age_citizens"] / df["total_pop"]

df = df.rename(columns = {
    "total_pop": "county_population",
}).drop(columns = ["population"])

df

,city_population,city_name,state_name,matched_city_name,city,city_ascii,state_id,county_fips,county_name,lat,...,mean_commute,employed,private_work,public_work,self_employed,family_work,unemployment_percentage,pct_men,pct_women,pct_voters
0,"8,336,817",New York,New York,New York,New York,New York,NY,36061,New York,40.6943,...,31.8,906389,83.8,8.3,7.8,0.2,6.2,0.472914,0.527086,0.715868
1,"3,979,576",Los Angeles,California,Los Angeles,Los Angeles,Los Angeles,CA,6037,Los Angeles,34.1139,...,30.9,4805817,79.3,11.2,9.3,0.2,7.8,0.492755,0.507245,0.615323
2,"462,628",Long Beach,California,Long Beach,Long Beach,Long Beach,CA,6037,Los Angeles,33.7981,...,30.9,4805817,79.3,11.2,9.3,0.2,7.8,0.492755,0.507245,0.615323
3,"212,979",Santa Clarita,California,Santa Clarita,Santa Clarita,Santa Clarita,CA,6037,Los Angeles,34.4175,...,30.9,4805817,79.3,11.2,9.3,0.2,7.8,0.492755,0.507245,0.615323
4,"199,303",Glendale,California,Glendale,Glendale,Glendale,CA,6037,Los Angeles,34.1818,...,30.9,4805817,79.3,11.2,9.3,0.2,7.8,0.492755,0.507245,0.615323
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
787,"49,688",Enid,Oklahoma,Enid,Enid,Enid,OK,40047,Garfield,36.4061,...,16.6,27796,80.7,12.7,6.4,0.1,5.3,0.495458,0.504542,0.700694
788,"49,154",Troy,New York,Troy,Troy,Troy,NY,36083,Rensselaer,42.7354,...,24.4,80468,73.5,21.8,4.6,0.2,5.8,0.493116,0.506884,0.778617
789,"48,115",Saginaw,Michigan,Saginaw,Saginaw,Saginaw,MI,26145,Saginaw,43.4199,...,21.7,82297,84.7,11.1,4.0,0.2,8.2,0.485116,0.514884,0.769952
790,"47,720",Niagara Falls,New York,Niagara Falls,Niagara Falls,Niagara Falls,NY,36063,Niagara,43.0921,...,22.0,100990,81.8,14.2,3.9,0.1,6.2,0.487091,0.512909,0.782339


In [ ]:
df.to_csv("df.csv")

## Merged data

In [2]:
df = pd.read_csv("./data/df.csv") # Columns are renamed and organized in Excel
df

,city,state_name,state_id,lat,lng,city_population,county_fips,county,county_population,density,...,pct_work_at_home,pct_private_work,pct_public_work,pct_self_employed,pct_family_work,avg_income,income_error,income_per_cap,income_per_cap_err,mean_commute
0,New York,New York,NY,40.6943,-73.9249,"8,336,817",36061,New York County,1653877,10715,...,6.7,83.8,8.3,7.8,0.2,79781,1158,69529,945,31.8
1,Los Angeles,California,CA,34.1139,-118.4068,"3,979,576",6037,Los Angeles County,10105722,3276,...,5.3,79.3,11.2,9.3,0.2,61015,262,30798,129,30.9
2,Long Beach,California,CA,33.7981,-118.1675,"462,628",6037,Los Angeles County,10105722,3523,...,5.3,79.3,11.2,9.3,0.2,61015,262,30798,129,30.9
3,Santa Clarita,California,CA,34.4175,-118.4964,"212,979",6037,Los Angeles County,10105722,1162,...,5.3,79.3,11.2,9.3,0.2,61015,262,30798,129,30.9
4,Glendale,California,CA,34.1818,-118.2468,"199,303",6037,Los Angeles County,10105722,2525,...,5.3,79.3,11.2,9.3,0.2,61015,262,30798,129,30.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
787,Enid,Oklahoma,OK,36.4061,-97.8701,"49,688",40047,Garfield County,62421,259,...,2.8,80.7,12.7,6.4,0.1,50724,1975,25787,960,16.6
788,Troy,New York,NY,42.7354,-73.6751,"49,154",36083,Rensselaer County,159800,1832,...,3.4,73.5,21.8,4.6,0.2,63166,1695,33067,710,24.4
789,Saginaw,Michigan,MI,43.4199,-83.9501,"48,115",26145,Saginaw County,193803,1087,...,3.5,84.7,11.1,4.0,0.2,45034,1104,25348,595,21.7
790,Niagara Falls,New York,NY,43.0921,-79.0147,"47,720",36063,Niagara County,212675,1308,...,2.6,81.8,14.2,3.9,0.1,51656,1205,28395,548,22.0


In [ ]:
df

,city,state_name,state_id,lat,lng,city_population,county_fips,county,county_population,density,...,pct_work_at_home,pct_private_work,pct_public_work,pct_self_employed,pct_family_work,avg_income,income_error,income_per_cap,income_per_cap_err,mean_commute
0,New York,New York,NY,40.6943,-73.9249,"8,336,817",36061,New York County,1653877,10715,...,6.7,83.8,8.3,7.8,0.2,79781,1158,69529,945,31.8
1,Los Angeles,California,CA,34.1139,-118.4068,"3,979,576",6037,Los Angeles County,10105722,3276,...,5.3,79.3,11.2,9.3,0.2,61015,262,30798,129,30.9
2,Long Beach,California,CA,33.7981,-118.1675,"462,628",6037,Los Angeles County,10105722,3523,...,5.3,79.3,11.2,9.3,0.2,61015,262,30798,129,30.9
3,Santa Clarita,California,CA,34.4175,-118.4964,"212,979",6037,Los Angeles County,10105722,1162,...,5.3,79.3,11.2,9.3,0.2,61015,262,30798,129,30.9
4,Glendale,California,CA,34.1818,-118.2468,"199,303",6037,Los Angeles County,10105722,2525,...,5.3,79.3,11.2,9.3,0.2,61015,262,30798,129,30.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
787,Enid,Oklahoma,OK,36.4061,-97.8701,"49,688",40047,Garfield County,62421,259,...,2.8,80.7,12.7,6.4,0.1,50724,1975,25787,960,16.6
788,Troy,New York,NY,42.7354,-73.6751,"49,154",36083,Rensselaer County,159800,1832,...,3.4,73.5,21.8,4.6,0.2,63166,1695,33067,710,24.4
789,Saginaw,Michigan,MI,43.4199,-83.9501,"48,115",26145,Saginaw County,193803,1087,...,3.5,84.7,11.1,4.0,0.2,45034,1104,25348,595,21.7
790,Niagara Falls,New York,NY,43.0921,-79.0147,"47,720",36063,Niagara County,212675,1308,...,2.6,81.8,14.2,3.9,0.1,51656,1205,28395,548,22.0


In [4]:
covid_by_county

,date,county,state,county_fips,cases,deaths,confirmed_cases,confirmed_deaths,probable_cases,probable_deaths
0,2020-10-17,Autauga,Alabama,1001.0,1966,28.0,1751.0,27.0,215.0,1.0
1,2020-10-17,Baldwin,Alabama,1003.0,6333,67.0,5462.0,63.0,871.0,4.0
2,2020-10-17,Barbour,Alabama,1005.0,968,9.0,702.0,9.0,266.0,0.0
3,2020-10-17,Bibb,Alabama,1007.0,771,13.0,721.0,9.0,50.0,4.0
4,2020-10-17,Blount,Alabama,1009.0,1783,23.0,1375.0,23.0,408.0,0.0
...,...,...,...,...,...,...,...,...,...,...
3240,2020-10-17,Sweetwater,Wyoming,56037.0,410,2.0,389.0,NaN,21.0,NaN
3241,2020-10-17,Teton,Wyoming,56039.0,692,1.0,659.0,NaN,33.0,NaN
3242,2020-10-17,Uinta,Wyoming,56041.0,392,2.0,317.0,NaN,75.0,NaN
3243,2020-10-17,Washakie,Wyoming,56043.0,132,7.0,122.0,NaN,10.0,NaN


In [5]:
df2 = pd.merge(
    df,
    covid_by_county,
    left_on = "county_fips",
    right_on = "county_fips"
)

In [6]:
cases_per_100k = (df2.cases / df2.county_population) * 100000 
deaths_per_100k = (df2.deaths / df2.county_population) * 100000

df2["covid_cases_per_100k"] = cases_per_100k
df2["covid_deaths_per_100k"] = deaths_per_100k
df2["covid_case_to_death_pct"] = (deaths_per_100k / cases_per_100k) * 100
df2[["city", "cases", "county_population", "covid_cases_per_100k", "covid_deaths_per_100k", "covid_case_to_death_pct"]]



df["covid_cases_per_100k"] = df2["covid_cases_per_100k"]
df["covid_deaths_per_100k"] = df2["covid_deaths_per_100k"]
df["covid_case_to_death_pct"] = df2["covid_case_to_death_pct"]
#df = df.drop(['confirmed_cases','confirmed_deaths','probable_cases','probable_deaths'], axis=1)


In [7]:
weather = pd.read_csv('data/county_weather_data.csv')
weather.head()

,state,county_fips,Location,LTM_mean_temp,LTM_mean_percipitation,LTM_min_temp,LTM_max_temp
0,AL,1001,Autauga County,65.9,69.15,54.8,77.0
1,AL,1003,Baldwin County,68.9,61.75,59.2,78.6
2,AL,1005,Barbour County,66.2,74.08,55.4,76.9
3,AL,1007,Bibb County,63.9,69.49,52.7,75.2
4,AL,1009,Blount County,62.7,76.46,52.6,72.6


In [8]:
with_weather = pd.merge(df,weather,left_on = "county_fips", right_on = "county_fips")
with_weather.rename(columns={'cases' : 'covid_cases', 'deaths' : 'covid_deaths'}, inplace=True)
df = with_weather
df

,city,state_name,state_id,lat,lng,city_population,county_fips,county,county_population,density,...,mean_commute,covid_cases_per_100k,covid_deaths_per_100k,covid_case_to_death_pct,state,Location,LTM_mean_temp,LTM_mean_percipitation,LTM_min_temp,LTM_max_temp
0,New York,New York,NY,40.6943,-73.9249,"8,336,817",36061,New York County,1653877,10715,...,31.8,2842.171989,67.832857,2.386656,NY,New York County,56.4,46.90,48.6,64.1
1,Los Angeles,California,CA,34.1139,-118.4068,"3,979,576",6037,Los Angeles County,10105722,3276,...,30.9,2842.171989,67.832857,2.386656,CA,Los Angeles County,62.9,16.61,51.4,74.4
2,Long Beach,California,CA,33.7981,-118.1675,"462,628",6037,Los Angeles County,10105722,3523,...,30.9,2842.171989,67.832857,2.386656,CA,Los Angeles County,62.9,16.61,51.4,74.4
3,Santa Clarita,California,CA,34.4175,-118.4964,"212,979",6037,Los Angeles County,10105722,1162,...,30.9,2842.171989,67.832857,2.386656,CA,Los Angeles County,62.9,16.61,51.4,74.4
4,Glendale,California,CA,34.1818,-118.2468,"199,303",6037,Los Angeles County,10105722,2525,...,30.9,2842.171989,67.832857,2.386656,CA,Los Angeles County,62.9,16.61,51.4,74.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
785,Enid,Oklahoma,OK,36.4061,-97.8701,"49,688",40047,Garfield County,62421,259,...,16.6,638.297872,26.282854,4.117647,OK,Garfield County,59.3,31.78,47.3,71.2
786,Troy,New York,NY,42.7354,-73.6751,"49,154",36083,Rensselaer County,159800,1832,...,24.4,1724.431510,76.366207,4.428486,NY,Rensselaer County,48.5,43.38,38.6,58.3
787,Saginaw,Michigan,MI,43.4199,-83.9501,"48,115",26145,Saginaw County,193803,1087,...,21.7,911.249559,42.318091,4.643963,MI,Saginaw County,49.2,39.00,39.8,58.5
788,Niagara Falls,New York,NY,43.0921,-79.0147,"47,720",36063,Niagara County,212675,1308,...,22.0,1729.783258,53.772887,3.108649,NY,Niagara County,49.5,33.02,41.2,57.8


In [ ]:
df.to_csv('test.csv')

In [13]:
cities_length = len(df)

def get_summary(row):
    print("Matching {}/{}".format(row.name, cities_length))
    full_city_name = row["city"] + " " + row["state_name"]
    try:
        summary = wikipedia.summary(full_city_name, sentences=4)
    except:
        return "No description available."

    print(full_city_name, summary)
    return summary

df["wikipedia_summary"] = df.apply(get_summary, axis = "columns")

Passaic New Jersey Passaic ( pə-SAY-ik or locally  pə-SAYK) is a city in Passaic County, New Jersey, United States. As of the 2010 United States Census, the city had a total population of 69,781, maintaining its status as the 15th largest municipality in New Jersey with an increase of 1,920 residents (+2.8%) from the 2000 Census population of 67,861, which had in turn increased by 9,820 (+16.9%) from the 58,041 counted in the 1990 Census.
Passaic is the tenth most densely populated municipality in the entire United States with 22,000+ people per square mile.
Located north of Newark on the Passaic River, it was first settled in 1678 by Dutch traders, as Acquackanonk Township.
Matching 429/790
Savannah Georgia Savannah () is the oldest city in the U.S. state of Georgia and is the county seat of Chatham County. Established in 1733 on the Savannah River, the city of Savannah became the British colonial capital of the Province of Georgia and later the first state capital of Georgia. A strat

In [14]:
df

,city,state_name,state_id,lat,lng,city_population,county_fips,county,county_population,density,...,covid_cases_per_100k,covid_deaths_per_100k,covid_case_to_death_pct,state,Location,LTM_mean_temp,LTM_mean_percipitation,LTM_min_temp,LTM_max_temp,wikipedia_summary
0,New York,New York,NY,40.6943,-73.9249,"8,336,817",36061,New York County,1653877,10715,...,2842.171989,67.832857,2.386656,NY,New York County,56.4,46.90,48.6,64.1,"New York City (NYC), often called simply New Y..."
1,Los Angeles,California,CA,34.1139,-118.4068,"3,979,576",6037,Los Angeles County,10105722,3276,...,2842.171989,67.832857,2.386656,CA,Los Angeles County,62.9,16.61,51.4,74.4,Los Angeles ( (listen); Spanish: Los Ángeles; ...
2,Long Beach,California,CA,33.7981,-118.1675,"462,628",6037,Los Angeles County,10105722,3523,...,2842.171989,67.832857,2.386656,CA,Los Angeles County,62.9,16.61,51.4,74.4,Long Beach is a city in the U.S. state of Cali...
3,Santa Clarita,California,CA,34.4175,-118.4964,"212,979",6037,Los Angeles County,10105722,1162,...,2842.171989,67.832857,2.386656,CA,Los Angeles County,62.9,16.61,51.4,74.4,Santa Clarita () is the third-largest city in ...
4,Glendale,California,CA,34.1818,-118.2468,"199,303",6037,Los Angeles County,10105722,2525,...,2842.171989,67.832857,2.386656,CA,Los Angeles County,62.9,16.61,51.4,74.4,"Glendale is a city in Los Angeles County, Cal..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
785,Enid,Oklahoma,OK,36.4061,-97.8701,"49,688",40047,Garfield County,62421,259,...,638.297872,26.282854,4.117647,OK,Garfield County,59.3,31.78,47.3,71.2,Enid ( EE-nid) is the ninth-largest city in th...
786,Troy,New York,NY,42.7354,-73.6751,"49,154",36083,Rensselaer County,159800,1832,...,1724.431510,76.366207,4.428486,NY,Rensselaer County,48.5,43.38,38.6,58.3,Troy is a city in the U.S. state of New York a...
787,Saginaw,Michigan,MI,43.4199,-83.9501,"48,115",26145,Saginaw County,193803,1087,...,911.249559,42.318091,4.643963,MI,Saginaw County,49.2,39.00,39.8,58.5,Saginaw () is a city in the U.S. state of Mich...
788,Niagara Falls,New York,NY,43.0921,-79.0147,"47,720",36063,Niagara County,212675,1308,...,1729.783258,53.772887,3.108649,NY,Niagara County,49.5,33.02,41.2,57.8,"Niagara Falls is a city in Niagara County, New..."


In [15]:
df.to_csv("df_w_wiki.csv")

In [49]:
df

,city,state_name,state_id,lat,lng,city_population,county_fips,county,county_population,density,...,covid_cases_per_100k,covid_deaths_per_100k,covid_case_to_death_pct,state,Location,LTM_mean_temp,LTM_mean_percipitation,LTM_min_temp,LTM_max_temp,wikipedia_summary
0,New York,New York,NY,40.6943,-73.9249,"8,336,817",36061,New York County,1653877,10715,...,2842.171989,67.832857,2.386656,NY,New York County,56.4,46.90,48.6,64.1,"New York City (NYC), often called simply New Y..."
1,Los Angeles,California,CA,34.1139,-118.4068,"3,979,576",6037,Los Angeles County,10105722,3276,...,2842.171989,67.832857,2.386656,CA,Los Angeles County,62.9,16.61,51.4,74.4,Los Angeles ( (listen); Spanish: Los Ángeles; ...
2,Long Beach,California,CA,33.7981,-118.1675,"462,628",6037,Los Angeles County,10105722,3523,...,2842.171989,67.832857,2.386656,CA,Los Angeles County,62.9,16.61,51.4,74.4,Long Beach is a city in the U.S. state of Cali...
3,Santa Clarita,California,CA,34.4175,-118.4964,"212,979",6037,Los Angeles County,10105722,1162,...,2842.171989,67.832857,2.386656,CA,Los Angeles County,62.9,16.61,51.4,74.4,Santa Clarita () is the third-largest city in ...
4,Glendale,California,CA,34.1818,-118.2468,"199,303",6037,Los Angeles County,10105722,2525,...,2842.171989,67.832857,2.386656,CA,Los Angeles County,62.9,16.61,51.4,74.4,"Glendale is a city in Los Angeles County, Cal..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
784,Enid,Oklahoma,OK,36.4061,-97.8701,"49,688",40047,Garfield County,62421,259,...,638.297872,26.282854,4.117647,OK,Garfield County,59.3,31.78,47.3,71.2,Enid ( EE-nid) is the ninth-largest city in th...
785,Troy,New York,NY,42.7354,-73.6751,"49,154",36083,Rensselaer County,159800,1832,...,1724.431510,76.366207,4.428486,NY,Rensselaer County,48.5,43.38,38.6,58.3,Troy is a city in the U.S. state of New York a...
786,Saginaw,Michigan,MI,43.4199,-83.9501,"48,115",26145,Saginaw County,193803,1087,...,911.249559,42.318091,4.643963,MI,Saginaw County,49.2,39.00,39.8,58.5,Saginaw () is a city in the U.S. state of Mich...
787,Niagara Falls,New York,NY,43.0921,-79.0147,"47,720",36063,Niagara County,212675,1308,...,1729.783258,53.772887,3.108649,NY,Niagara County,49.5,33.02,41.2,57.8,"Niagara Falls is a city in Niagara County, New..."


In [50]:
voting = pd.read_csv('voting_data.csv')
voting = voting.drop(['state','county','white_pct','black_pct','hispanic_pct','nonwhite_pct','female_pct','median_hh_inc'],axis=1)
voting['rep_pct'] = voting.trump16/(voting.trump16 + voting.clinton16 + voting.otherpres16)
voting['dem_pct'] = voting.clinton16/(voting.trump16 + voting.clinton16 + voting.otherpres16)


In [72]:
df = pd.read_csv('df_w_wiki.csv',index_col=0)

df = pd.merge(df,voting,left_on='county_fips',right_on='fips')
df = df.drop(['state','Location','fips','trump16','clinton16','otherpres16','total_population'], axis=1)
df


,city,state_name,state_id,lat,lng,city_population,county_fips,county,county_population,density,...,age65andolder_pct,clf_unemploy_pct,lesshs_pct,lesscollege_pct,lesshs_whites_pct,lesscollege_whites_pct,rural_pct,ruralurban_cc,rep_pct,dem_pct
0,New York,New York,NY,40.6943,-73.9249,"8,336,817",36061,New York County,1653877,10715,...,14.383705,6.869787,12.996284,39.565360,1.784064,17.134644,0.000000,1.0,0.097063,0.865569
1,Los Angeles,California,CA,34.1139,-118.4068,"3,979,576",6037,Los Angeles County,10105722,3276,...,12.202278,8.908614,22.282470,69.235895,5.681371,51.917013,0.605218,1.0,0.224133,0.717572
2,Long Beach,California,CA,33.7981,-118.1675,"462,628",6037,Los Angeles County,10105722,3523,...,12.202278,8.908614,22.282470,69.235895,5.681371,51.917013,0.605218,1.0,0.224133,0.717572
3,Santa Clarita,California,CA,34.4175,-118.4964,"212,979",6037,Los Angeles County,10105722,1162,...,12.202278,8.908614,22.282470,69.235895,5.681371,51.917013,0.605218,1.0,0.224133,0.717572
4,Glendale,California,CA,34.1818,-118.2468,"199,303",6037,Los Angeles County,10105722,2525,...,12.202278,8.908614,22.282470,69.235895,5.681371,51.917013,0.605218,1.0,0.224133,0.717572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
784,Enid,Oklahoma,OK,36.4061,-97.8701,"49,688",40047,Garfield County,62421,259,...,15.415886,4.946972,12.766375,78.643327,9.287302,76.710624,21.411357,5.0,0.737402,0.202533
785,Troy,New York,NY,42.7354,-73.6751,"49,154",36083,Rensselaer County,159800,1832,...,15.103245,6.754134,8.667169,70.689358,7.912411,70.415625,30.512015,2.0,0.471284,0.457184
786,Saginaw,Michigan,MI,43.4199,-83.9501,"48,115",26145,Saginaw County,193803,1087,...,17.154625,9.535561,11.528984,79.273610,8.976539,77.011599,31.118205,3.0,0.482072,0.470696
787,Niagara Falls,New York,NY,43.0921,-79.0147,"47,720",36063,Niagara County,212675,1308,...,17.268739,7.139954,9.123949,76.505515,8.230073,75.986947,22.440627,1.0,0.562336,0.384829


In [85]:
taxes["county"].value_counts()

Washington County    28
Franklin County      23
Jefferson County     23
Jackson County       19
Lincoln County       18
                     ..
Broward County        1
Harvey County         1
Obion County          1
Montcalm County       1
Kauai County          1
Name: county, Length: 1556, dtype: int64

In [92]:
taxes = pd.read_csv('data/Taxes/taxes_df.csv')
#taxes = taxes.drop(['state_name'],axis=1)
taxes
df2 = pd.merge(df,taxes,how='left',left_on=['county','state_id'],right_on=['county','state_name'])
df = df2


In [75]:
crime = pd.read_csv('processed_crime_final.csv')
df = pd.merge(df,crime,on=['city','state_name'])
df

,city,state_name,state_id,lat,lng,city_population,county_fips,county,county_population,density,...,clf_unemploy_pct,lesshs_pct,lesscollege_pct,lesshs_whites_pct,lesscollege_whites_pct,rural_pct,ruralurban_cc,rep_pct,dem_pct,crime_per_100
0,New York,New York,NY,40.6943,-73.9249,"8,336,817",36061,New York County,1653877,10715,...,6.869787,12.996284,39.565360,1.784064,17.134644,0.000000,1.0,0.097063,0.865569,0.036231
1,Los Angeles,California,CA,34.1139,-118.4068,"3,979,576",6037,Los Angeles County,10105722,3276,...,8.908614,22.282470,69.235895,5.681371,51.917013,0.605218,1.0,0.224133,0.717572,0.053825
2,Long Beach,California,CA,33.7981,-118.1675,"462,628",6037,Los Angeles County,10105722,3523,...,8.908614,22.282470,69.235895,5.681371,51.917013,0.605218,1.0,0.224133,0.717572,0.066151
3,Santa Clarita,California,CA,34.4175,-118.4964,"212,979",6037,Los Angeles County,10105722,1162,...,8.908614,22.282470,69.235895,5.681371,51.917013,0.605218,1.0,0.224133,0.717572,0.029470
4,Glendale,California,CA,34.1818,-118.2468,"199,303",6037,Los Angeles County,10105722,2525,...,8.908614,22.282470,69.235895,5.681371,51.917013,0.605218,1.0,0.224133,0.717572,0.035725
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
696,Battle Creek,Michigan,MI,42.2986,-85.2296,"51,093",26025,Calhoun County,134327,463,...,8.456289,9.428094,79.249181,8.712921,78.917095,30.977039,3.0,0.534685,0.410122,0.080781
697,Florissant,Missouri,MO,38.7996,-90.3269,"50,952",29189,St. Louis County,999539,1567,...,6.696711,7.049887,57.617910,4.973065,52.119318,1.137990,1.0,0.393184,0.556860,0.046894
698,Joplin,Missouri,MO,37.0758,-94.5018,"50,925",29097,Jasper County,118522,515,...,6.013659,12.968027,77.943574,10.368353,77.136641,23.691697,3.0,0.725696,0.218764,0.143410
699,Enid,Oklahoma,OK,36.4061,-97.8701,"49,688",40047,Garfield County,62421,259,...,4.946972,12.766375,78.643327,9.287302,76.710624,21.411357,5.0,0.737402,0.202533,0.075323


In [93]:
df.to_csv('final_dataset.csv')